In [1]:
from rich import print

from example_results import result_with_chapters

transcript = result_with_chapters.timestamped_transcript
print(transcript)

[00:05] [Music] Hi, I'm Leela with NVIDIA, and this is the Jetson AGX Thor Developer Kit.
[00:05] Jetson Thor is the ultimate platform for humanoid robotics.
[00:13] It's part of NVIDIA's three-computer solution for accelerating physical AI: NVIDIA DGX for training, NVIDIA
Omniverse for synthetic data generation and physical AI simulation, and NVIDIA Jetson Thor for runtime robotics.
[00:26] Jetson AGX Thor is the ideal runtime computer for any kind of physical AI application, from humanoids to 
Edge AI agents.
[00:40] It gives you unmatched performance and scalability in a compact, power-efficient form factor.
[00:46] Plus, it's powered by the advanced Blackwell GPU and 128 GB of memory, delivering up to 2070 FP4 TFLOPS of 
AI compute to effortlessly run the latest generative AI models.
[00:54] Let's take a closer look at the Jetson AGX Thor Developer Kit.
[01:02] It includes a Jetson T5000 module, a reference carrier board, an active heat sink with a fan, and a power 
supply.
[01:09] If you want to set up with a monitor, you'll need to provide a computer display, keyboard, and mouse to get
started.
[01:22] To make your setup process as smooth as possible, we've conveniently placed all the I/O ports on one side 
of the board.
[01:29] This thoughtful design choice makes it super easy to connect your sensors and manage your wiring when 
building your robot.
[01:34] Starting here, we see two USB-A ports, an Ethernet port, a DisplayPort, an HDMI port, two USB-C ports, a 
QSFP slot, and a Micro-Fit port for extending the board's power if needed.
[01:50] Under the developer kit, we have an M.
[01:50] 2 key M slot with a populated 1TB NVMe storage and an M.
[01:57] 2 key E slot with a pre-installed wireless networking card.
[02:05] On the other side of the developer kit, there are three buttons for power, force recovery, and reset.
[02:11] It's easy to get started.
[02:11] In the box is a small booklet that includes a link to step-by-step instructions.
[02:19] First, download the latest NVIDIA JetPack 7 ISO image from the JetPack SDK download page and flash it to a 
USB drive to install on your developer kit.
[02:30] After attaching your monitor, keyboard, and mouse, just connect the power supply and your developer kit 
will power on automatically.
[02:37] On first boot, you'll be prompted to choose a username, password, and other basic information.
[02:43] You can also set up your Wi-Fi at this time.
[02:48] Jetson Thor delivers a seamless cloud-to-edge experience by running the NVIDIA AI software stack for 
physical AI applications.
[02:57] This includes NVIDIA Isaac for robotics, NVIDIA Metropolis for video analytics AI agents, and NVIDIA 
Holoscan for sensor processing.
[03:04] With up to 7.
[03:04] 5x the performance of Jetson AGX Orin, Jetson Thor can run all modern AI models from vision-language-action
models like NVIDIA Isaac GR00T N1 to all popular LLMs and VLMs.
[03:19] NVIDIA Isaac GR00Tis redefining the future of robotics with four key pillars: robotic foundational models, 
synthetic data pipelines, a simulation environment, and a runtime computer.
[03:34] You can also accelerate your software development by tethering the developer kit directly to any existing 
robot.
[03:45] Here, NVIDIA Isaac GR00T N1 is running on Thor.
[03:51] To show a developer's end-to-end journey, we've fine-tuned GR00T N1 for picking up a nut and pouring it in 
Isaac Sim and Isaac Lab.
[03:58] We've also implemented a hardware-in-the-loop scenario running NVIDIA Omniverse on RTX Pro 6000 and 
inferencing on Jetson Thor.
[04:06] The AI blueprint for Video Search and Summarization (VSS) from NVIDIA Metropolis gives you the tools to 
build and deploy video analytics AI agents that can perform contextualized real-time alerts, video summarization, 
and Q&A by analyzing live camera streams.
[04:26] VSS is powering visual agent workforces spanning many use cases, including visual inspection and worker 
safety in manufacturing, fan engagement and player

In [2]:
"""
This module provides functions for processing transcribed text to generate formatted subtitles and AI-powered summaries using the Gemini API.
"""

import os

from dotenv import load_dotenv
from google.genai import Client, types
from pydantic import BaseModel, Field

from youtube_summarizer.utils import is_youtube_url

load_dotenv()


class Chapter(BaseModel):
    header: str = Field(description="A descriptive title for the chapter")
    key_points: list[str] = Field(description="Important takeaways and insights from this chapter")
    summary: str = Field(description="A comprehensive summary of the chapter content")


class Analysis(BaseModel):
    title: str = Field(description="The main title or topic of the video content")
    summary: str = Field(description="A comprehensive summary of the video content")
    takeaways: list[str] = Field(description="Key insights and actionable takeaways for the audience")
    key_facts: list[str] = Field(description="Important facts, statistics, or data points mentioned")
    chapters: list[Chapter] = Field(description="Structured breakdown of content into logical chapters")
    keywords: list[str] = Field(description="Keywords or topics mentioned in the video, max 3", max_length=3)


def summarize_video(url_or_transcript: str) -> Analysis:
    """
    Summarize the text using the Gemini. Streaming seems to be less buggy with long videos.
    """
    if is_youtube_url(url_or_transcript):
        print(f"🔗 Sending YouTube URL to Gemini: {url_or_transcript}")
    else:
        print(f"📝 Sending transcript text to Gemini: {len(url_or_transcript)} characters")
        print(f"📝 Text preview: {url_or_transcript[:200]}...")

    client = Client(
        api_key=os.getenv("GEMINI_API_KEY"),
        http_options={"timeout": 600000},
    )

    response = client.models.generate_content(
        model="models/gemini-2.5-pro",
        contents=types.Content(
            parts=[
                types.Part(file_data=types.FileData(file_uri=url_or_transcript)) if is_youtube_url(url_or_transcript) else types.Part(text=url_or_transcript),
            ]
        ),
        config=types.GenerateContentConfig(
            system_instruction="""Analyze the video/transcript according to the schema.
The transcript provides starting timestamps for each sentence.
Add the timestamps [TIMESTAMP] at the end of the takeaways and key facts if available.
Consider the chapters (headers) if given but not necessary.
Ignore the promotional and meaningless content.
Follow the original language.""",
            temperature=0,
            response_mime_type="application/json",
            response_schema=Analysis,
            thinking_config=types.ThinkingConfig(thinking_budget=2048),
        ),
    )

    print(response.usage_metadata)

    return response.parsed


result = summarize_video(transcript)
print(result)

📝 Sending transcript text to Gemini: 5292 characters

📝 Text preview: [00:05] [Music] Hi, I'm Leela with NVIDIA, and this is the Jetson AGX Thor Developer Kit.
[00:05] Jetson Thor is the ultimate platform for humanoid robotics.
[00:13] It's part of NVIDIA's three-comput...

GenerateContentResponseUsageMetadata(
    cache_tokens_details=None,
    cached_content_token_count=None,
    candidates_token_count=1501,
    candidates_tokens_details=None,
    prompt_token_count=1513,
    prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=1513)],
    thoughts_token_count=885,
    tool_use_prompt_token_count=None,
    tool_use_prompt_tokens_details=None,
    total_token_count=3899,
    traffic_type=None
)

Analysis(
    title='An Overview of the NVIDIA Jetson AGX Thor Developer Kit',
    summary="This video introduces the NVIDIA Jetson AGX Thor Developer Kit, positioning it as the ultimate 
platform for humanoid robotics and physical AI. It details its role within NVIDIA's three-computer solution 
alongside DGX and Omniverse. The presentation covers the hardware specifications, including the Blackwell GPU and 
2070 FP4 TFLOPS of AI compute, and provides a tour of the developer kit's contents and I/O ports. The video also 
guides users through the initial setup process using the JetPack 7 SDK. Finally, it showcases the platform's 
capabilities by demonstrating its integration with NVIDIA's software stacks like Isaac for robotics (running GR00T 
N1), Metropolis for video analytics AI agents (VSS), and Holoscan for real-time sensor processing.",
    takeaways=[
        'Jetson AGX Thor is the ideal runtime computer for physical AI applications, from humanoids to Edge AI 
agents. [00:26]',
        "It is part of NVIDIA's comprehensive solution for physical AI, which includes NVIDIA DGX for training and 
NVIDIA Omniverse for simulation. [00:13]",
        'The developer kit is designed for ease of use, with all I/O ports conveniently placed on one side for 
simple sensor connection and wire management. [01:22]',
        'Getting started involves downloading the NVIDIA JetPack 7 ISO image and flashing it to a USB drive to 
install on the developer kit. [02:19]',
        'The platform runs the NVIDIA AI software stack, enabling advanced applications in robotics (Isaac), video 
analytics (Metropolis), and sensor processing (Holoscan). [02:57]',
        'Jetson Thor can run modern AI models, including vision-language-action models like NVIDIA Isaac GR00T N1, 
and popular LLMs and VLMs. [03:04]'
    ],
    key_facts=[
        'Powered by the advanced Blackwell GPU and 128 GB of memory. [00:46]',
        'Delivers up to 2070 FP4 TFLOPS of AI compute. [00:46]',
        'The developer kit includes a Jetson T5000 module, a reference carrier board, an active heat sink, and a 
power supply. [01:02]',
        'The board features two USB-A ports, an Ethernet port, a DisplayPort, an HDMI port, two USB-C ports, a QSFP
slot, and a Micro-Fit power port. [01:34]',
        'Comes with a populated 1TB NVMe storage in its M.2 key M slot and a pre-installed wireless networking card
in the M.2 key E slot. [01:50]',
        'Offers up to 7.5x the performance of the previous generation Jetson AGX Orin. [03:04]'
    ],
    chapters=[
        Chapter(
            header='Introduction to Jetson AGX Thor',
            key_points=[
                'Jetson Thor is presented as the ultimate platform for humanoid robotics and physical AI. [00:05]',
                "It is a key component of NVIDIA's three-computer solution for accelerating physical AI. [00:13]",
                'It features the Blackwell GPU, 128 GB of memory, and delivers up to 2070 FP4 TFLOPS of AI compute.
[00:46]'
            ],
            summary="This chapter introduces the Jetson AGX Thor as a powerful runtime computer for physical AI. It
highlights its position within NVIDIA's ecosystem for training (DGX), simulation (Omniverse), and runtime (Jetson).
The segment emphasizes its high performance, powered by the Blackwell GPU, making it capable of running the latest 
generative AI models."
        ),
        Chapter(
            header='Developer Kit Hardware Overview',
            key_points=[
                'The developer kit includes the T5000 module, carrier board, heat sink, and power supply. [01:02]',
                'All I/O ports are conveniently located on one side of the board for easy access and wiring. 
[01:22]',
                'The kit includes 1TB of NVMe storage and a pre-installed wireless networking card. [01:50]'
            ],
            summary='This section provides a detailed look at the Jetson AGX Thor Developer Kit. It outlines the 
components included in the box and 